In [13]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
from sqlalchemy import create_engine
import re
from MTIPmapping import *

In [2]:
path = r'T:\MPO\TIP\TIP FY24-27\Maps\Data'
points = gpd.read_file(os.path.join(path, 'FY24_27_points.shp'))
lines = gpd.read_file(os.path.join(path, 'FY24_27_lines.shp'))

In [3]:
[k for k in points.STIP_Key.unique() if k in lines.STIP_Key.unique()]

['22348']

In [4]:
points.loc[points.STIP_Key=='22348', 'ID'].unique()

array([14], dtype=int64)

In [5]:
data = pd.read_excel(r'T:\MPO\TIP\TIP FY24-27\Maps\MTIP_FY24-27_20230303.xlsx')

In [6]:
data.shape

(190, 19)

In [7]:
Ids = points[['STIP_Key', 'ID']].drop_duplicates().append(lines[['STIP_Key', 'ID']].drop_duplicates(), ignore_index=True)
Ids.rename(columns={'STIP_Key':'STIP Key'}, inplace=True)

In [8]:
data.columns

Index(['Geo', 'Project Name', 'MTIP ID #', 'Project Description', 'Work Type',
       'Perf. Meas.', 'RTP Project Number / Ref.', 'Air Quality Status',
       'STIP Key', 'FFY', 'Phase', 'Federal Funding',
       'Federal Funding Source ', 'Federal Req. Match',
       'Federal Req. Match Source', 'Total Fed+ Req Match', 'Other Funding',
       'Other Funding Source', 'Total All Sources'],
      dtype='object')

In [9]:
Ids['STIP Key'] = Ids['STIP Key'].astype(np.int64)

In [10]:
ndata = data.merge(Ids, how='outer', on='STIP Key')

In [11]:
23058 in ndata['STIP Key'].unique()

True

In [12]:
sdata = ndata[ndata['STIP Key'] != 23058]

In [17]:
sdata.loc[:,'AQ Exempt?'] = sdata.loc[:,'Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Exempt?')) 
sdata.loc[:,'AQ Status'] = sdata.loc[:,'Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Status'))
sdata.loc[:,'IAC'] = sdata.loc[:,'Air Quality Status'].apply(lambda x: getAQvars(x, var='IAC')) 

In [19]:
pointdata = get_district_IDs(dat=points, shptype='point', keycol='STIP_Key', 
                             colnm1='Senator District', colnm2='Representative District')
linedata = get_district_IDs(dat=lines, shptype='line', keycol='STIP_Key', 
                             colnm1='Senator District', colnm2='Representative District')

In [21]:
df = pointdata.append(linedata , ignore_index=True)
df.rename(columns={'STIP_Key':'STIP Key'}, inplace=True)
df['STIP Key'] = df['STIP Key'].astype(np.int64)

In [22]:
ndata = sdata.merge(df, how='outer', on='STIP Key')

In [23]:
ndata.shape

(222, 25)

In [24]:
ndata.to_excel(os.path.join(path, 'MTIP24_27_List_20230303.xlsx'), 
               sheet_name='2024-2027 Project List', 
               index=False)  